# Descargador de archivos a Google Drive

Este notebook permite descargar archivos desde URLs y guardarlos en Google Drive.

In [ ]:
# Instalación de dependencias necesarias
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client requests ipywidgets

In [ ]:
import os
import io
import requests
import pickle
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseUpload
import ipywidgets as widgets
from IPython.display import display, HTML
from urllib.parse import urlparse

In [ ]:
# Configuración de autenticación de Google Drive
SCOPES = ['https://www.googleapis.com/auth/drive.file']

def get_google_credentials(client_secrets):
    creds = None
    
    # Si ya hay credenciales guardadas, usarlas
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    
    # Si no hay credenciales válidas, solicitar al usuario que inicie sesión
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_config(
                client_secrets,
                SCOPES
            )
            creds = flow.run_local_server(port=0)
        
        # Guardar las credenciales para la próxima vez
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    
    return creds

In [ ]:
# Crear widgets para la interfaz de usuario
client_config_input = widgets.Textarea(
    description='Credenciales de Google:',
    placeholder='Pega aquí el contenido del archivo client_secrets.json',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='70%', height='150px')
)

url_input = widgets.Text(
    description='URL del archivo:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='70%')
)

folder_name_input = widgets.Text(
    description='Carpeta en Drive:',
    value='Descargas',  # Carpeta por defecto
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

status_output = widgets.Output()

download_button = widgets.Button(
    description='Descargar',
    button_style='success'
)

In [ ]:
def get_or_create_folder(service, folder_name):
    # Buscar si la carpeta ya existe
    query = f"name='{folder_name}' and mimeType='application/vnd.google-apps.folder' and trashed=false"
    results = service.files().list(q=query, spaces='drive', fields='files(id, name)').execute()
    folders = results.get('files', [])
    
    # Si la carpeta existe, retornar su ID
    if folders:
        return folders[0]['id']
    
    # Si no existe, crear una nueva carpeta
    folder_metadata = {
        'name': folder_name,
        'mimeType': 'application/vnd.google-apps.folder'
    }
    folder = service.files().create(body=folder_metadata, fields='id').execute()
    return folder['id']

def download_to_drive(b):
    with status_output:
        status_output.clear_output()
        try:
            # Obtener credenciales
            import json
            client_secrets = json.loads(client_config_input.value)
            creds = get_google_credentials(client_secrets)
            
            # Crear servicio de Google Drive
            service = build('drive', 'v3', credentials=creds)
            
            # Obtener o crear carpeta de destino
            folder_id = get_or_create_folder(service, folder_name_input.value)
            
            # Obtener la URL y el nombre del archivo
            url = url_input.value
            filename = os.path.basename(urlparse(url).path)
            if not filename:
                filename = 'downloaded_file'
            
            # Descargar el archivo
            print(f'📥 Descargando {filename}...')
            response = requests.get(url)
            response.raise_for_status()
            
            # Preparar el archivo para subir a Drive
            file_metadata = {
                'name': filename,
                'parents': [folder_id]
            }
            
            media = MediaIoBaseUpload(
                io.BytesIO(response.content),
                mimetype=response.headers.get('content-type', 'application/octet-stream'),
                resumable=True
            )
            
            # Subir a Google Drive
            print('📤 Subiendo a Google Drive...')
            file = service.files().create(
                body=file_metadata,
                media_body=media,
                fields='id'
            ).execute()
            
            print(f'✅ Archivo guardado exitosamente en la carpeta {folder_name_input.value}')
            
        except Exception as e:
            print(f'❌ Error: {str(e)}')

# Asociar la función al botón
download_button.on_click(download_to_drive)

In [ ]:
# Mostrar la interfaz
display(HTML('<h3>Configuración de Google Drive</h3>'))
display(client_config_input)
display(HTML('<h3>Información del archivo</h3>'))
display(url_input)
display(folder_name_input)
display(download_button)
display(status_output)